In [ ]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain-groq mysql-connector-python python-dotenv

In [ ]:
import os
import pprint
from dotenv import load_dotenv

# Verify the current working directory
print("Current working directory:", os.getcwd())
load_dotenv()

MYSQL_HOST = os.getenv("MYSQL_HOST", "localhost")
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "")
MYSQL_DB = os.getenv("MYSQL_DB", "test_db")
MYSQL_PORT = os.getenv("cat MYSQL_PORT", "3306")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(os.getcwd())
print(os.environ)
print(GROQ_API_KEY)

Part 1: Build a Basic Chatbot¶

We'll first create a simple chatbot using LangGraph. This chatbot will respond directly to user messages. Though simple, it will illustrate the core concepts of building with LangGraph. By the end of this section, you will have a built rudimentary chatbot.

Start by creating a StateGraph. A StateGraph object defines the structure of our chatbot as a "state machine". We'll add nodes to represent the llm and functions our chatbot can call and edges to specify how the bot should transition between these functions.

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

Next, add a "chatbot" node. Nodes represent units of work. They are typically regular python functions

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model_name="llama-3.1-8b-instant")


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("posts", chatbot)

In [ ]:
graph_builder.add_edge(START, "posts")
graph_builder.add_edge("posts", END)
graph = graph_builder.compile()

Query para generar la setencia sql

In [ ]:
def generate_sql_query(question, schema):
    print(schema)

    prompt = f"""
    Below is the schema for a MySQL database. Generate an SQL query based on the user's question:

    Database schema:
    {schema}

    User's question: {question}

    Please return only the SQL query, nothing else. I only require the SQL.
    """

    response = llm.invoke(prompt)
    # Token usage
    pprint.pp(f"Token usage: {response.usage_metadata}")

    response = response.content.replace("```sql", "").replace("```", "")

    return response

Connection to the DB

In [ ]:
import mysql.connector
def connect_to_database():
    conn = mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DB,
        port=MYSQL_PORT
    )
    return conn

Get the schemas of the tables

In [ ]:
def get_database_schema():
    tables = os.getenv("TABLES", "posts")
    table_list = [table.strip() for table in tables.split(",")]
    conn = connect_to_database()
    cursor = conn.cursor()
    schema = {}
    for table in table_list:
        cursor.execute(f"DESCRIBE {table}")
        result = cursor.fetchall()
        schema[table] = result
    cursor.close()
    conn.close()
    return str(schema)

Make the request to the DB

In [ ]:
def run_sql_query(query):
    conn = connect_to_database()
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    return result

Interact with the LLM

In [ ]:


def generate_natural_language_response(question, query, result):
    llmguard = ChatGroq(temperature=0, model_name="llama-3.1-8b-instant")
    prompt = f"""
        Below is the schema for a MySQL database and the SQL query generated to answer a question:

        Database schema:
        {get_database_schema()}

        User's question: {question}
        SQL query: {query}
        Result: {result}

        Write a response in natural language for the user based on the query and the result. It is important to include the post URL and social network if post information is requested.
        """

    response = llmguard.invoke(prompt)

    # Token usage
    pprint.pp(f"Token usage: {response.usage_metadata}")

    return response.content


In [ ]:

# Main function to interact with the agent
def interact_with_agent():
    print('enters 1')
    question = input("Ask a question about the database: ")
    print(question)
    if question.lower() == "exit":
        print("Exiting...")
        return
    print('enters 2')
    schema = get_database_schema()
    print('enters 3')
    query = generate_sql_query(question, schema)
    print(f"Generated query: {query}")

    result = run_sql_query(query)
    print(f"Query result: {result}")

    response = generate_natural_language_response(question, query, result)
    print(f"Natural language response: {response}")

    interact_with_agent()

if __name__ == "__main__":
    print("Hello! I'm your SQL query agent.")
    interact_with_agent()